In [1]:
import os
import pickle
import scipy.io
import numpy as np

from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
results_folder = './results/'

In [3]:
data_folder = '../data'

problem_name = 'syn_1000_1500_10_50_3'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [4]:
results_file=os.path.join(results_folder,'portfolio',problem_name+'.pckl')
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results=pickle.load(f)
        except Exception: # so many things could go wrong, can't be more specific.
            pass 

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-11

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
    'rho':np.sqrt(n), #parameters for ll00
    'diam_X':np.sqrt(2),
    'sigma_f':1,                   
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [6]:
func_x = lambda x, dot_product: pr.portfolio(R,x,dot_product)
func_beta = lambda x, s, beta, dot_product, dot_product_s: pr.portfolio(R,(1 - beta) * x + beta * s,(1 - beta) * dot_product + beta * dot_product_s)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
prox_func = lambda x, L: pr.proj_simplex(x)

In [7]:
points = [x0]
 
for _ in range(1):
    x = np.abs(np.random.rand(len(x0)))
    x = x / sum(x)
    points.append(x)

## Run FW

In [8]:
run_alpha_policies = ['new_lloo', 'backtracking']

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x, 
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    
with open(results_file, 'wb') as f:
        pickle.dump(results, f)

********* Algorithm starts *********
5.402714639414114
iter = 1, stepsize = 0.38445711349830225, criterion = 0.09999999999999999, upper_bound=-0.02331708937379639, lower_bound=-5.42603172878791, real_Gap=5.402714639414114
iter = 1000, stepsize = 0.0012083386271202028, criterion = 0.001140853464774229, upper_bound=-4.8162024201217974, lower_bound=-4.8164952384270725, real_Gap=0.0002928183052750555
iter = 2000, stepsize = 0.0007223813286357581, criterion = 0.0005596979503138339, upper_bound=-4.81620588487121, lower_bound=-4.81630489760676, real_Gap=9.901273555001922e-05
iter = 3000, stepsize = 0.0010014827275507804, criterion = 0.00037072658219996645, upper_bound=-4.81620647052039, lower_bound=-4.8162874856226185, real_Gap=8.101510222857655e-05
iter = 4000, stepsize = 0.0007503396585714417, criterion = 0.0002771682164169367, upper_bound=-4.816206698504663, lower_bound=-4.8162635824941695, real_Gap=5.688398950631779e-05
iter = 5000, stepsize = 0.0006000206744381555, criterion = 0.00022145

In [ ]:
run_alpha_policies = ["backtracking","standard","line_search","icml"]

for policy in run_alpha_policies:
    print(policy)
    #policy_results = []
    #for i in range(len(points)):
    #    x0 = points[i]
    x, alpha_hist, Gap_hist, Q_hist, time_hist = frank_wolfe(func_x,
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,                                       
                        alpha_policy=policy,                                                    
                        eps=terminate_tol,
                        print_every=1000, 
                        debug_info=False)
        
    results[problem_name][policy]={
    'x': x,
    'alpha_hist': alpha_hist,        
    'Q_hist': Q_hist,
    'Gap_hist': Gap_hist,        
    'time_hist': time_hist,}
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
#    results[problem_name][policy] = policy_results

backtracking
********* Algorithm starts *********
iter = 1, stepsize = 7.363777025964322e-08, criterion = 7.361322024385784e-08, upper_bound=0.02128931286912572, lower_bound=-4.917163298675437, real_Gap=4.938452611544562
iter = 1000, stepsize = 0.001052323689082904, criterion = 9.617941984714084e-10, upper_bound=-4.457535820063278, lower_bound=-4.477659316146569, real_Gap=0.02012349608329167
iter = 2000, stepsize = 7.874479898687913e-06, criterion = 9.617941984714084e-10, upper_bound=-4.466570818394181, lower_bound=-4.47572472164284, real_Gap=0.00915390324865939
iter = 3000, stepsize = 0.0011121441717925045, criterion = 8.440960863353929e-10, upper_bound=-4.469641752388953, lower_bound=-4.475554081171069, real_Gap=0.005912328782115672
iter = 4000, stepsize = 7.477168899465519e-06, criterion = 8.440960863353929e-10, upper_bound=-4.471098118413302, lower_bound=-4.475269162757174, real_Gap=0.004171044343872055
iter = 5000, stepsize = 0.0003174547338831935, criterion = 8.440960863353929e-1

## Run SCOPT

In [ ]:
x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
        grad_x,
        hess_mult_x,
        hess_mult_vec_x,
        Mf,
        nu,
        prox_func,
        x0,  
        sc_params,                                              
        eps=terminate_tol,                                              
        print_every=1)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
#    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

Fista err = 9.939e-06; Subiter = 119; subproblem converged!

iter =    1, stepsize = 5.194e-01, rdiff = 5.792e-01 , f = 0.0302781

iter =    2, stepsize = 6.923e-01, rdiff = 2.797e-01 , f = -2.3233

iter =    3, stepsize = 8.807e-01, rdiff = 8.542e-02 , f = -3.71065

Fista err = 9.884e-06; Subiter =  15; subproblem converged!

iter =    4, stepsize = 9.845e-01, rdiff = 9.939e-03 , f = -4.22803

Fista err = 5.447e-06; Subiter =   3; subproblem converged!

iter =    5, stepsize = 9.998e-01, rdiff = 1.299e-04 , f = -4.29596

Fista err = 1.479e-08; Subiter =   2; subproblem converged!

iter =    6, stepsize = 1.000e+00, rdiff = 1.128e-06 , f = -4.29703

Fista err = 2.801e-08; Subiter =   2; subproblem converged!

iter =    7, stepsize = 1.000e+00, rdiff = 5.706e-08 , f = -4.29703

Fista err = 1.050e-10; Subiter =   2; subproblem converged!

iter =    8, stepsize = 1.000e+00, rdiff = 3.481e-09 , f = -4.29703

Fista err = 3.194e-11; Subiter =   2; subproblem converged!

iter =    9, stepsize

In [ ]:
with open(results_file, 'wb') as f:
    pickle.dump(results, f)